In [1]:
import pickle
import Feature_Extraction
import pandas as pd

haar family: haar
db family: db1, db2, db3, db4, db5, db6, db7, db8, db9, db10, db11, db12, db13, db14, db15, db16, db17, db18, db19, db20, db21, db22, db23, db24, db25, db26, db27, db28, db29, db30, db31, db32, db33, db34, db35, db36, db37, db38
sym family: sym2, sym3, sym4, sym5, sym6, sym7, sym8, sym9, sym10, sym11, sym12, sym13, sym14, sym15, sym16, sym17, sym18, sym19, sym20
coif family: coif1, coif2, coif3, coif4, coif5, coif6, coif7, coif8, coif9, coif10, coif11, coif12, coif13, coif14, coif15, coif16, coif17
bior family: bior1.1, bior1.3, bior1.5, bior2.2, bior2.4, bior2.6, bior2.8, bior3.1, bior3.3, bior3.5, bior3.7, bior3.9, bior4.4, bior5.5, bior6.8
rbio family: rbio1.1, rbio1.3, rbio1.5, rbio2.2, rbio2.4, rbio2.6, rbio2.8, rbio3.1, rbio3.3, rbio3.5, rbio3.7, rbio3.9, rbio4.4, rbio5.5, rbio6.8
dmey family: dmey
gaus family: gaus1, gaus2, gaus3, gaus4, gaus5, gaus6, gaus7, gaus8
mexh family: mexh
morl family: morl
cgau family: cgau1, cgau2, cgau3, cgau4, cgau5, cgau6, cgau7, 

In [2]:
fe = Feature_Extraction.FeatureExtraction()

In [3]:
def load_in_participant_data(path):
    with open(path, 'rb') as pickle_file:
        loaded_data = pickle.load(pickle_file)

    NeedToSearch = loaded_data['NeedToSearch']
    CorrectSearch = loaded_data['CorrectSearch']
    IncorrectSearch = loaded_data['IncorrectSearch']

    return NeedToSearch, CorrectSearch, IncorrectSearch

In [4]:
import numpy as np

def align_features_to_matrix(feature_dict):
    # Get the number of frequency bands
    num_bands = len(list(feature_dict.keys()))

    # Get the number of channels and samples for one frequency band
    channels, samples = feature_dict[list(feature_dict.keys())[0]].shape

    # Initialize the aligned matrix
    aligned_matrix = np.zeros((samples, channels * num_bands))

    # Iterate through frequency bands and populate the matrix
    for idx, (band, power) in enumerate(feature_dict.items()):
        start_col = idx * channels
        end_col = (idx + 1) * channels
        aligned_matrix[:, start_col:end_col] = power.T

    return aligned_matrix



In [5]:
def search_condition_feature_dataframe(SearchFeatures):
    feature_set = []
    if len(SearchFeatures[0]) == 0:
        return 0
    for i in range(len(SearchFeatures[0])):
        EEG_features = align_features_to_matrix(SearchFeatures[0][i])
        feature_set.append(EEG_features)

    return feature_set, SearchFeatures[1]

In [6]:
ParticipantList = ["01", "02", "03", "04", "05", "06", "08", "09", "10"]

In [7]:
Participant_df_dict = {}

for participant in ParticipantList:
    path = fr"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Information-Need\Data\Oscillatory_Features\participant_0{participant}_Features.pkl"

    NeedToSearch, CorrectSearch, IncorrectSearch = load_in_participant_data(path)

    if participant != "06":
        NeedToSearch_df = search_condition_feature_dataframe(NeedToSearch)
        #print(f"NeedToSearch_df: {len(NeedToSearch_df[0])}")
    CorrectSearch_df = search_condition_feature_dataframe(CorrectSearch)
    #print(f"CorrectSearch_df: {len(CorrectSearch_df[0])}")
    IncorrectSearch_df = search_condition_feature_dataframe(IncorrectSearch)
    #print(f"IncorrectSearch_df: {len(IncorrectSearch_df[0])}")

    Participant_df_dict[participant] = [NeedToSearch_df, CorrectSearch_df, IncorrectSearch_df]



EOFError: Ran out of input

In [151]:
def create_all_participant_df(loaded_dictionary):
    NeedToSearch_feature = []
    CorrectSearch_feature = []
    IncorrectSearch_feature = []

    for key, value in Participant_df_dict.items():
        print('Participant:', key)
        NeedToSearch = value[0][0]
        for EEG in NeedToSearch:
            NeedToSearch_feature.append(pd.DataFrame(EEG))

        CorrectSearch = value[1][0]
        for EEG in CorrectSearch:
            CorrectSearch_feature.append(pd.DataFrame(EEG))

        IncorrectSearch = value[2][0]
        for EEG in IncorrectSearch:
            IncorrectSearch_feature.append(pd.DataFrame(EEG))

    return NeedToSearch_feature, CorrectSearch_feature, IncorrectSearch_feature

In [152]:
NeedToSearch_features, CorrectSearch_features, IncorrectSearch_features = create_all_participant_df(Participant_df_dict)

Participant: 01
Participant: 02
Participant: 03
Participant: 04
Participant: 05
Participant: 06
Participant: 08
Participant: 09
Participant: 10


In [156]:
NeedToSearch_features_df = pd.concat(NeedToSearch_features)
CorrectSearch_features_df = pd.concat(CorrectSearch_features)
IncorrectSearch_features_df = pd.concat(IncorrectSearch_features)

In [160]:
NeedToSearch_features_df['Label'] = 0
NeedToSearch_features_df = NeedToSearch_features_df.dropna()
CorrectSearch_features_df['Label'] = 1
CorrectSearch_features_df = CorrectSearch_features_df.dropna()
IncorrectSearch_features_df['Label'] = 1
IncorrectSearch_features_df = IncorrectSearch_features_df.dropna()

MemoryError: Unable to allocate 2.36 GiB for an array with shape (328, 964800) and data type float64

In [159]:
X_CorrectSearch = CorrectSearch_features_df.sample(n=int(len(NeedToSearch_features_df)/2), random_state=1)
X_IncorrectSearch = CorrectSearch_features_df.sample(n=int(len(NeedToSearch_features_df)/2), random_state=1)

TypeError: expected a sequence of integers or a single integer, got '254800.0'